## Example -- cn-ocr model visualization and conversion

##### In this example, we visualize and convert a pre-trained Mxnet model to keras using the sbs_mxnet2keras package. The model file is from cn-ocr https://github.com/breezedeus/cnocr, can be accessed in the data folder.

##### Load the package

In [1]:
import os.path
import mxnet2keras
from mxnet2keras.Weight_Converter import WeightConvert
from mxnet2keras.Model_Summary import ModelSummary

##### Creating visualization plot by Model_Summary.plot()

In [11]:
data_path = os.path.join(mxnet2keras.__path__[0], 'data')
summary_ocr_model = ModelSummary(os.path.join(data_path, 'cnocr-v1.2.0-conv-lite-fc'), 25) # input: model prefix + epoch
summary_ocr_model.plot('pdf') #input: type, can also be change to 'png'
# visualization can be accessed in the current directory: graph.pd

graph.pdf can be accsssed within current directory


##### Create summary of the model by Model_Summary.summary()

In [12]:
summary_ocr_model.summary()

________________________________________________________________________________________________________________________
Layer (type)                                        Output Shape            Param #     Previous Layer                  
data(null)                                                                  0                                           
________________________________________________________________________________________________________________________
conv-0(Convolution)                                                         64          data                            
________________________________________________________________________________________________________________________
batchnorm-0(BatchNorm)                                                      0           conv-0                          
________________________________________________________________________________________________________________________
leakyrelu-0(LeakyReLU)          

##### Create a list of layer names by Model_Summary.get_layer_names()

In [13]:
summary_ocr_model.get_layer_names()

['seq-fc',
 'batchnorm-1',
 'batchnorm-4',
 'conv-5-2-1x1',
 'batchnorm-6',
 'conv-1',
 'conv-3',
 'conv-6',
 'conv-6-2-1x1',
 'conv-5',
 'conv-4-1-1x1',
 'batchnorm-0',
 'batchnorm-5',
 'conv-0',
 'batchnorm-2',
 'conv-5-1-1x1',
 'conv-2',
 'conv-4',
 'conv-6-1-1x1',
 'pred_fc',
 'batchnorm-3',
 'conv-4-2-1x1']

##### Create a list of layer names filtered by layer types by Model_Summary.filtered_layer_names()

In [14]:
summary_ocr_model.filtered_layer_names('convolution')

['conv-5-2-1x1',
 'conv-1',
 'conv-3',
 'conv-6',
 'conv-6-2-1x1',
 'conv-5',
 'conv-4-1-1x1',
 'conv-0',
 'conv-5-1-1x1',
 'conv-2',
 'conv-4',
 'conv-6-1-1x1',
 'conv-4-2-1x1']

In [15]:
summary_ocr_model.filtered_layer_names('conv') # either 'conv' or 'convolution' could get to the result

['conv-5-2-1x1',
 'conv-1',
 'conv-3',
 'conv-6',
 'conv-6-2-1x1',
 'conv-5',
 'conv-4-1-1x1',
 'conv-0',
 'conv-5-1-1x1',
 'conv-2',
 'conv-4',
 'conv-6-1-1x1',
 'conv-4-2-1x1']

In [16]:
summary_ocr_model.filtered_layer_names('batchnorm') # currently support for: fully connected, batchnorm, convolution, and reLu

['batchnorm-1',
 'batchnorm-4',
 'batchnorm-6',
 'batchnorm-0',
 'batchnorm-5',
 'batchnorm-2',
 'batchnorm-3']

#### Define keras model Structure

In [17]:
# with the help of the visualizations, users can mannually define the keras net structure
# here, I mannually defined the model structure in keras based on visualizations above.
from mxnet2keras.tests.build_net_for_test import build_net
keras_model = build_net()

#### Laoding Weight with Weight_Converter

In [18]:
loadweight_ocr_model = WeightConvert(os.path.join(data_path, 'cnocr-v1.2.0-conv-lite-fc'), 25, keras_model) #input: model_prefix, epoch, keras model input

#### Loading single layer Weight with load_single_weight()

In [19]:
loadweight_ocr_model.load_single_weight('conv-1') # input: layer name

#### Check if the weight loaded correctly

In [22]:
weight_mx = loadweight_ocr_model.arg_params['conv-1' + "_bias"].asnumpy()
layer = loadweight_ocr_model.keras_net.get_layer('conv-1')
weight_keras = layer.get_weights()[1]
print(max(weight_keras-weight_mx))

0.0


#### Loading weights to a type of layers with load_type_weight()

In [23]:
loadweight_ocr_model.load_type_weight('convolution') # input: layer name

In [24]:
weight_mx = loadweight_ocr_model.arg_params['conv-2' + "_bias"].asnumpy()
layer = loadweight_ocr_model.keras_net.get_layer('conv-2')
weight_keras = layer.get_weights()[1]
print(max(weight_keras-weight_mx))

0.0


#### Loading weights to a all layers

In [25]:
loadweight_ocr_model.load_all_weight()

In [27]:
gamma_mx = loadweight_ocr_model.arg_params['batchnorm-0' +"_gamma"].asnumpy()
beta_mx = loadweight_ocr_model.arg_params['batchnorm-0' + "_beta"].asnumpy()
layer = loadweight_ocr_model.keras_net.get_layer('batchnorm-0')
gamma_keras, beta_keras = layer.get_weights()[:2]
print(max(gamma_keras-gamma_mx))
print(max(beta_keras-beta_mx))

0.0
0.0
